In [1]:
import tensorflow as tf
import numpy as np
import math
import keras
from keras.callbacks import *
import os
from pathlib import Path
import cv2
import keras.backend as K

model_file = 'keras_model.h5'
lite_file = 'keras_model.tflite'

num_pictures=490623
num_classes=10572
batch_size=32
target_size=(112,112)


In [2]:
def my_generator(tfrecord_path='C:/Users/mengf/workspace/MobileFaceNet_TF_i/datasets/faces_webface_112x112/tfrecords/tran.tfrecords',batch_size=32,out_num=10572):
    """自定义generator
    
    # Argument
        tfrecord_path:
        batch_size
        out_num: 类别数量，用于生成onehot
        
    # Return
    
    """
    def parse_function(example_proto):
        features = {'image_raw': tf.FixedLenFeature([], tf.string),
                    'label': tf.FixedLenFeature([], tf.int64)}
        features = tf.parse_single_example(example_proto, features)
        # You can do more image distortion here for training data
        img = tf.image.decode_jpeg(features['image_raw'])
        img = tf.reshape(img, shape=(112, 112, 3))

        #img = tf.py_func(random_rotate_image, [img], tf.uint8)
        img = tf.cast(img, dtype=tf.float32)
        img = tf.subtract(img, 127.5)
        img = tf.multiply(img,  0.0078125)
        img = tf.image.random_flip_left_right(img)
        label = tf.cast(features['label'], tf.int64)
        label = tf.one_hot(label,out_num)
        return img, label
    
#     config = tf.ConfigProto(allow_soft_placement=True)
#     sess = tf.Session(config=config)
    sess = K.get_session()
    # training datasets api config
    tfrecords_f = os.path.join(tfrecord_path)
    dataset = tf.data.TFRecordDataset(tfrecords_f)
    dataset = dataset.map(parse_function)
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()
    # begin iteration
    while(True):
        sess.run(iterator.initializer)
        while True:
            try:
                images, labels = sess.run(next_element)
                for i in range(len(images)):
                    images[i,...] = cv2.cvtColor(images[i, ...], cv2.COLOR_RGB2BGR)
                yield images,labels
            except tf.errors.OutOfRangeError:
#                 print("End of dataset")
                pass

def my_generator_wrapper():
    for image,label in my_generator():
        yield ([image,label],label)

In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout,PReLU,Layer
from keras.layers import Activation, BatchNormalization, add, Reshape,DepthwiseConv2D
from keras.utils.vis_utils import plot_model
from keras.activations import relu
from keras.initializers import Constant
from keras import regularizers

from keras import backend as K

weight_decay = 1e-4  # l2正则化decay常量

def flow_wrapper(flow):
    """自定义wrapper，将(x,y)变成([x,y],y)"""
    while True:
        x,y = flow.next()
        yield ([x,y],y)

def prelu(input, name=''):
    """自定义prelu"""
    alphas = K.variable(K.constant(0.25,dtype=tf.float32,shape=[input.get_shape()[-1]]),name=name + 'prelu_alphas')
    pos = K.relu(input)
    neg = alphas * (input - K.abs(input)) * 0.5
    return pos + neg

cval = Constant(0.25)  # prelu α 初始常量

def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides, kernel_initializer='glorot_normal',)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    x = PReLU(cval)(x)
#     x = Activation(relu)(x)
    return x


def _bottleneck(inputs, filters, kernel, t, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        r: Boolean, Whether to use the residuals.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same', kernel_initializer='glorot_normal')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = PReLU(cval)(x)
#     x = Activation(relu)(x)

    x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same', kernel_initializer='glorot_normal')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = add([x, inputs])
    return x


def _inverted_residual_block(inputs, filters, kernel, t, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """

    x = _bottleneck(inputs, filters, kernel, t, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, 1, True)

    return x


class ArcFace(Layer):
    """改进的softmax，得出的结果再与真是结果之间求交叉熵"""
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs # x为embeddings，y为labels
        c = K.shape(x)[-1]  # 特征维度
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        # 全连接层，x的结构为（None，128）w的结构为（128，n_classes）。logits的结构为(None,n_classes)
        # (np.random.randn(5,128) @ np.random.randn(128,10)).shape # (5, 10)
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

def MobileFaceNets(input_shape=(112,112,3), n_classes=10, k=128):
    """MobileFaceNets"""
    inputs = Input(shape=input_shape) #112x112，(img-127.5)/255
    y      = Input(shape=(n_classes,))
    x = _conv_block(inputs, 64, (3, 3), strides=(2, 2))
    
    
    
    # depthwise conv3x3
    x = DepthwiseConv2D(3, strides=(1, 1), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU(cval)(x)
#     x = Activation(relu)(x)
    
    
    
    # 5层bottleneck
    x = _inverted_residual_block(x, 64, (3, 3), t=2, strides=2, n=5)
    x = _inverted_residual_block(x, 128, (3, 3), t=4, strides=2, n=1)
    x = _inverted_residual_block(x, 128, (3, 3), t=2, strides=1, n=6)
    x = _inverted_residual_block(x, 128, (3, 3), t=4, strides=2, n=1)
    x = _inverted_residual_block(x, 128, (3, 3), t=2, strides=1, n=2)
    
    
    
    # conv1x1
    x = _conv_block(x, 512, (1, 1), strides=(1, 1))
    
    
    
    # linear GDConv7x7
    x = DepthwiseConv2D(7, strides=(1, 1), depth_multiplier=1, padding='valid')(x)
    x = Dropout(0.3, name='Dropout')(x)
    
    
    
    x = Conv2D(k, (1, 1), padding='same')(x)
    x = Reshape((k,))(x)
    # x 为embeddings， y为embeddings对应的类别标签，output为
    output = ArcFace(n_classes=n_classes, regularizer=regularizers.l2(weight_decay))([x, y])
    
    model = Model([inputs, y], output)
#     plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)
    print(model.input,model.output)
    return model



model = MobileFaceNets(n_classes=num_classes)
model.compile(optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'])

callbacks = [
    # 当监测值不再改善时，该回调函数将中止训练
    # 如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练。
    EarlyStopping(monitor='loss', patience=10, verbose=1),
    # 该回调函数将日志信息写入TensorBorad
#     TensorBoard(log_dir='./models/logs',histogram_freq=1),
    # 当评价指标不在提升时，减少学习率
    # min_lr：学习率的下限
    ReduceLROnPlateau(monitor='loss',factor=0.2,patience=3,min_lr=0.0001),
    # 该回调函数将在每个epoch后保存模型到filepath
    ModelCheckpoint(filepath='models/weights-autoencoder-{epoch:02d}-{loss:.2f}.h5',save_best_only=True)
]

model.fit_generator(my_generator_wrapper(),steps_per_epoch=num_pictures//batch_size,epochs=100,callbacks=callbacks)

In [7]:
def preprocess(img):
    """图片预处理，the image is substracted 127.5 and multiplied 1/128."""
    return (img-127.5)*0.0078125

datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess,)

flow = datagen.flow_from_directory('/workspace/dataset/face_ms1m1/',target_size=(112,112),batch_size=90,)

flow.next()[0].shape,flow.n,flow.num_classes

Found 68543 images belonging to 1005 classes.


((90, 112, 112, 3), 68543, 1005)

In [54]:
def load_img_from_directory(path='/workspace/dataset/face_ms1m1/'):
    """从path中加载图片的标签和路径"""
    p = Path(path)
    # 结构为图片的标签和图片的绝对路径
    rets = []
    
    # 图片父目录文件夹名对应的id
    label_dict = {}
    
    for i,file in enumerate(p.glob('*')):
        file_name=str(file.name)
        label_dict[file_name]=i
    
    for i,file in enumerate(p.glob('*/*')):
        # 获得图片的父目录名字
        parent = str(file.parent.name)
        # 图片绝对路径
        file_path = str(file)
        # 重新设置标签
        index = label_dict[parent]
        rets.append([index,file_path])
    return rets, label_dict

def my_generator(images,labels,batch_size=32,preprocess_function=lambda x:(x-127.5)*0.0078125,shuffle=False,target_size=(112,112)):
    """图片和标签生成器，images为路径，label为标签"""
    
    def load_img(imgs_path):
        imgs = []
        for img_path in imgs_path:
            img = tf.image.decode_image(img_path)
            img = tf.image.resize(img, target_size)
            img = preprocess_function(img)
            imgs.append(img)
        imgs = np.asarray(imgs)    
        return imgs
        
    
    num_imgs = len(images)
    # 52//32=1
    num_of_steps = num_imgs//batch_size
#     tf.image.flip_left_right()
    while True:
        for step in range(num_of_steps):
            left = step*batch_size
            right = (step+1)*batch_size
            batch_imgs = images[left:right]
            Image.open(batch_imgs[1])
            

In [55]:
imgs,label_dict= load_img_from_directory()

In [57]:
len(label_dict)

1005

In [4]:
path1 = '/workspace/dataset/face_ms1m/'
path2 = '/workspace/dataset/face_ms1m1/'
di = os.listdir('/workspace/dataset/face_ms1m/')
di1 = [path1 + x for x in di]
di2 = [path2 + x for x in di]

In [7]:
for i in range(1000):
    !cp -r {di1[i]} {di2[i]}

In [10]:
!ls -l {path1} | wc -l

1006


In [5]:
model = Model(inputs=model.inputs, outputs=model.layers[-3].output)

In [6]:
model.save(model_file)

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_file)
tflite_model = converter.convert()
open(lite_file, "wb").write(tflite_model)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 325 variables.
INFO:tensorflow:Converted 325 variables to const ops.


3923212

In [48]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=lite_file)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('INPUTS: ')
print(input_details)
print('OUTPUTS: ')
print(output_details)

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=input_details[0]['dtype'])
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print('output:')
print(output_data)

INPUTS: 
[{'name': 'input_1', 'index': 81, 'shape': array([  1, 112, 112,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
OUTPUTS: 
[{'name': 'reshape_1/Reshape', 'index': 115, 'shape': array([  1, 128], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
output:
[[ 2.04079999e-11  2.82742059e-11 -4.58211351e-11 -5.56394123e-14
  -3.91572851e-11 -1.02505678e-11 -2.42160354e-11  4.17224762e-12
  -1.25698618e-11  4.44564940e-11  3.76039894e-11 -2.25695504e-11
  -4.11849027e-11 -3.03026076e-11 -1.68697938e-11 -4.17013212e-12
   6.04923837e-12 -2.29401793e-11  2.21237317e-11 -2.08520666e-11
   1.48790737e-11 -9.07520933e-12 -3.46710993e-11  4.28729413e-11
  -2.50271748e-11 -2.58911746e-11  2.78507616e-11 -1.55599752e-11
  -8.76598620e-12 -3.71855706e-11  1.05097554e-11 -7.10245231e-12
  -5.50741310e-12 -3.52396862e-11  2.18869870e-11  1.15694555e-11
  -1.38298765e-11  7.41387247e-12 -2.91055756e-11 -4.24613643e-11
   2.57873133e-11 -3